In [14]:
import sys, random
# print(sys.executable)
# print(sys.version)
import cv2
import cvzone
import pyautogui
import mediapipe as mp
from cvzone.HandTrackingModule import HandDetector

In [2]:
def all_fingers_raised(lmList):

    if len(lmList) != 21:
        return False
        
    thumb_tip = 4
    thumb_ip = 3
    index_tip = 8
    index_ip = 7
    middle_tip = 12
    middle_ip = 11
    ring_tip = 16
    ring_ip = 15
    pinky_tip = 20
    pinky_ip = 19
    return (is_finger_raised(lmList, thumb_tip, thumb_ip) and
            is_finger_raised(lmList, index_tip, index_ip) and
            is_finger_raised(lmList, middle_tip, middle_ip) and
            is_finger_raised(lmList, ring_tip, ring_ip) and
            is_finger_raised(lmList, pinky_tip, pinky_ip))

def is_finger_raised(lmList, tip, phalanx):
    return lmList[tip][1] < lmList[phalanx][1]

In [7]:
def mouse_movement(lmList):
    index_finger = lmList[8][:2]
    middle_finger = lmList[12][:2]
    distance = ((index_finger[0] - middle_finger[0])**2 + (index_finger[1] - middle_finger[1])**2)**0.5
    x = int(index_finger[0] * screen_width / frame.shape[1])
    y = int(index_finger[1] * screen_height / frame.shape[0])

    # Move mouse cursor
    pyautogui.moveTo(x, y)

In [10]:
def set_camera():
    cap = cv2.VideoCapture(0)
    cap.set(3, screen_width)
    cap.set(4, screen_height)
    return cap

In [12]:

def run():

    hand_detector = HandDetector(detectionCon=0, maxHands=1)
    screen_width, screen_height = pyautogui.size()
    cap = set_camera()
    while cap.isOpened() and running:
        success, frame = cap.read()
        if not success:
            continue
    
        frame = cv2.flip(frame, 1)
        hands, img = hand_detector.findHands(frame)
    
        if hands: 
            lmList = hands[0]['lmList']
            
            # stops the program
            if all_fingers_raised(lmList):
                break
            
            mouse_movement(lmList)
    
    cap.release()
    cv2.destroyAllWindows()


In [13]:
run()

I0000 00:00:1726773866.414399   42094 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1726773866.416202   43454 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 Mesa 24.0.3-1pop1~1711635559~22.04~7a9f319), renderer: Mesa Intel(R) UHD Graphics 620 (WHL GT2)
W0000 00:00:1726773866.453489   43446 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1726773866.482540   43444 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
/home/austin/projects/drag-and-drop/venv/lib/python3.10/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '